In [ ]:
X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]
X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

NameError: name 'X_mnist' is not defined

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.datasets import mnist

# 1) Keras에서 MNIST 로드
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# 2) (n_samples, 28*28) 형태로 평탄화 + float 변환
X_train_full = X_train_full.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test       = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0
y_train_full = y_train_full.astype(np.int8)
y_test       = y_test.astype(np.int8)

# 3) train 50,000 / valid 10,000 으로 다시 나누기
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full,
    test_size=10000, random_state=42, stratify=y_train_full
)


# 3) 개별 모델 정의
clf1 = ExtraTreesClassifier(n_estimators=100, random_state=42)
clf2 = LinearSVC(max_iter=2000)
clf3 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=20, random_state=42)

# 4) 개별 모델 학습
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

# 5) 검증 세트에서 각 모델 정확도 비교
valid_pred1 = clf1.predict(X_valid)
valid_pred2 = clf2.predict(X_valid)
valid_pred3 = clf3.predict(X_valid)

print("ExtraTrees valid acc :", accuracy_score(y_valid, valid_pred1))
print("LinearSVC valid acc  :", accuracy_score(y_valid, valid_pred2))
print("MLP valid acc        :", accuracy_score(y_valid, valid_pred3))

# 6) Voting Classifier (Hard Voting)
voting_clf = VotingClassifier(
    estimators=[
        ('extra', clf1),
        ('svc', clf2),
        ('mlp', clf3)
    ],
    voting='hard'
)

# Voting 학습
voting_clf.fit(X_train, y_train)

# 7) 검증 세트 성능
valid_pred_vote = voting_clf.predict(X_valid)
print("Voting valid acc     :", accuracy_score(y_valid, valid_pred_vote))

# 8) 최종 Test 세트 성능 비교
test_pred1 = clf1.predict(X_test)
test_pred2 = clf2.predict(X_test)
test_pred3 = clf3.predict(X_test)
test_pred_vote = voting_clf.predict(X_test)

print("\n--- Test Accuracy ---")
print("ExtraTrees :", accuracy_score(y_test, test_pred1))
print("LinearSVC  :", accuracy_score(y_test, test_pred2))
print("MLP        :", accuracy_score(y_test, test_pred3))
print("Voting     :", accuracy_score(y_test, test_pred_vote))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


ExtraTrees valid acc : 0.9701
LinearSVC valid acc  : 0.9136
MLP valid acc        : 0.9754


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting valid acc     : 0.971

--- Test Accuracy ---
ExtraTrees : 0.9712
LinearSVC  : 0.9159
MLP        : 0.9749
Voting     : 0.9687


voting이 더 떨어져서 soft방법으로 해보기로 함

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# 1. MNIST 로드 & 전처리
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_train_full = X_train_full.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test       = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0
y_train_full = y_train_full.astype(np.int8)
y_test       = y_test.astype(np.int8)

# ---- 옵션: 학습을 빠르게 하려면 train 일부만 사용 (예: 20,000개) ----
X_train_sub, _, y_train_sub, _ = train_test_split(
    X_train_full, y_train_full,
    train_size=20000, random_state=42, stratify=y_train_full
)
X_train = X_train_sub
y_train = y_train_sub

# 검증용으로 10,000개 따로 떼고 싶으면:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train,
    test_size=5000, random_state=42, stratify=y_train
)

print("Train:", X_train.shape, "Valid:", X_valid.shape, "Test:", X_test.shape)

# 2. 개별 분류기 (둘 다 predict_proba 지원 → soft voting 가능)
clf_et = ExtraTreesClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

clf_mlp = MLPClassifier(
    hidden_layer_sizes=(100,),  # 작게 잡아서 속도↑
    max_iter=20,                # epoch 줄이기
    alpha=1e-4,
    random_state=42
)

# 3. 개별 모델 학습
print("\n[Train] ExtraTrees")
clf_et.fit(X_train, y_train)

print("\n[Train] MLP")
clf_mlp.fit(X_train, y_train)

# 4. 검증 세트에서 단일 모델 성능
valid_pred_et  = clf_et.predict(X_valid)
valid_pred_mlp = clf_mlp.predict(X_valid)

print("\n=== Validation Accuracy (single models) ===")
print("ExtraTrees :", accuracy_score(y_valid, valid_pred_et))
print("MLP        :", accuracy_score(y_valid, valid_pred_mlp))

# 5. Soft Voting 앙상블 (ExtraTrees + MLP)
voting_soft = VotingClassifier(
    estimators=[
        ('et',  clf_et),
        ('mlp', clf_mlp)
    ],
    voting='soft',      # 확률 평균
    n_jobs=-1
)

print("\n[Train] Soft Voting (ET + MLP)")
voting_soft.fit(X_train, y_train)

# 6. 검증 세트 성능
valid_pred_vote = voting_soft.predict(X_valid)
print("\n=== Validation Accuracy (Soft Voting) ===")
print("Soft Voting :", accuracy_score(y_valid, valid_pred_vote))

# 7. 테스트 성능
test_pred_et   = clf_et.predict(X_test)
test_pred_mlp  = clf_mlp.predict(X_test)
test_pred_vote = voting_soft.predict(X_test)

print("\n=== Test Accuracy ===")
print("ExtraTrees :", accuracy_score(y_test, test_pred_et))
print("MLP        :", accuracy_score(y_test, test_pred_mlp))
print("SoftVote   :", accuracy_score(y_test, test_pred_vote))


Train: (15000, 784) Valid: (5000, 784) Test: (10000, 784)

[Train] ExtraTrees

[Train] MLP


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(



=== Validation Accuracy (single models) ===
ExtraTrees : 0.9614
MLP        : 0.948

[Train] Soft Voting (ET + MLP)

=== Validation Accuracy (Soft Voting) ===
Soft Voting : 0.9544

=== Test Accuracy ===
ExtraTrees : 0.958
MLP        : 0.9559
SoftVote   : 0.9602


In [ ]:
# ================================
# 0. 라이브러리 임포트
# ================================
import numpy as np

from tensorflow.keras.datasets import mnist

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# ================================
# 1. MNIST 데이터 로드 & 전처리
# ================================
# (X_train_full, y_train_full): 60,000
# (X_test, y_test): 10,000
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# (N, 28, 28) -> (N, 784) + 정규화
X_train_full = X_train_full.reshape(-1, 28 * 28).astype(np.float32) / 255.0
X_test       = X_test.reshape(-1, 28 * 28).astype(np.float32) / 255.0
y_train_full = y_train_full.astype(np.int8)
y_test       = y_test.astype(np.int8)

# train 50,000 / valid 10,000으로 분리
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full,
    y_train_full,
    test_size=10000,
    random_state=42,
    stratify=y_train_full
)

print("Train:", X_train.shape, "Valid:", X_valid.shape, "Test:", X_test.shape)

# ================================
# 2. 개별 분류기 정의
# ================================
# 1) ExtraTrees (랜덤 포레스트 계열)
clf_et = ExtraTreesClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# 2) SVC (확률 출력 가능하도록 probability=True)
#   - LinearSVC는 predict_proba가 없어서 soft voting 불가 → SVC로 교체
clf_svc = SVC(
    kernel='linear',        # 과부하되면 'linear' 유지, 더 높이고 싶으면 'rbf'로 실험
    probability=True,       # soft voting을 위해 필수
    random_state=42
)

# 3) MLP (간단한 신경망)
clf_mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    max_iter=30,            # 필요하면 늘리기
    alpha=1e-4,
    random_state=42,
    verbose=False
)

# ================================
# 3. 개별 모델 학습
# ================================
print("\n[Train] ExtraTrees")
clf_et.fit(X_train, y_train)

print("\n[Train] SVC (probability=True)")
clf_svc.fit(X_train, y_train)

print("\n[Train] MLP")
clf_mlp.fit(X_train, y_train)

# ================================
# 4. 검증 세트에서 개별 성능 확인
# ================================
valid_pred_et  = clf_et.predict(X_valid)
valid_pred_svc = clf_svc.predict(X_valid)
valid_pred_mlp = clf_mlp.predict(X_valid)

print("\n=== Validation Accuracy (single models) ===")
print("ExtraTrees :", accuracy_score(y_valid, valid_pred_et))
print("SVC        :", accuracy_score(y_valid, valid_pred_svc))
print("MLP        :", accuracy_score(y_valid, valid_pred_mlp))

# ================================
# 5. Soft Voting 앙상블 정의 & 학습
# ================================
voting_soft = VotingClassifier(
    estimators=[
        ('et',  clf_et),
        ('svc', clf_svc),
        ('mlp', clf_mlp)
    ],
    voting='soft',          # <-- 핵심: soft voting
    n_jobs=-1
)

print("\n[Train] Soft VotingClassifier")
voting_soft.fit(X_train, y_train)

# ================================
# 6. 검증 세트에서 soft voting 성능
# ================================
valid_pred_vote = voting_soft.predict(X_valid)
print("\n=== Validation Accuracy (Soft Voting) ===")
print("Soft Voting :", accuracy_score(y_valid, valid_pred_vote))

# ================================
# 7. 최종 Test 세트 성능 비교
# ================================
test_pred_et    = clf_et.predict(X_test)
test_pred_svc   = clf_svc.predict(X_test)
test_pred_mlp   = clf_mlp.predict(X_test)
test_pred_vote  = voting_soft.predict(X_test)

print("\n=== Test Accuracy ===")
print("ExtraTrees :", accuracy_score(y_test, test_pred_et))
print("SVC        :", accuracy_score(y_test, test_pred_svc))
print("MLP        :", accuracy_score(y_test, test_pred_mlp))
print("SoftVote   :", accuracy_score(y_test, test_pred_vote))


Train: (50000, 784) Valid: (10000, 784) Test: (10000, 784)

[Train] ExtraTrees

[Train] SVC (probability=True)


In [2]:
# ==========================================
# 0. 라이브러리 임포트
# ==========================================
import numpy as np

from tensorflow.keras.datasets import mnist

from sklearn.ensemble import ExtraTreesClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# ==========================================
# 1. MNIST 데이터 로드 & 전처리
# ==========================================
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_train_full = X_train_full.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test       = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0
y_train_full = y_train_full.astype(np.int8)
y_test       = y_test.astype(np.int8)

print("Train:", X_train_full.shape, "Test:", X_test.shape)

# ==========================================
# 2. 개별 기본 모델(base estimators)
# ==========================================
clf_et = ExtraTreesClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

clf_svc = SVC(
    kernel='linear',
    probability=True,
    random_state=42
)

clf_mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    max_iter=30,
    alpha=1e-4,
    random_state=42
)

# ==========================================
# 3. StackingClassifier 구성
# ==========================================

estimators = [
    ('et', clf_et),
    ('svc', clf_svc),
    ('mlp', clf_mlp),
]

meta_model = LogisticRegression(max_iter=200)

stack_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    stack_method='predict_proba',  # 확률 기반 스태킹
    n_jobs=-1,
    passthrough=False              # 기본 모델 입력 피쳐를 메타 모델에 전달하지 않음
)

# ==========================================
# 4. 스태킹 모델 학습
# ==========================================
print("\n[Train] StackingClassifier")
stack_clf.fit(X_train_full, y_train_full)

# ==========================================
# 5. 테스트 성능 평가
# ==========================================
test_pred_stack = stack_clf.predict(X_test)
acc_stack = accuracy_score(y_test, test_pred_stack)

print("\n=== Test Accuracy ===")
print("Stacking :", acc_stack)


Train: (60000, 784) Test: (10000, 784)

[Train] StackingClassifier


KeyboardInterrupt: 

In [3]:
# ==========================================
# 0. 라이브러리 임포트
# ==========================================
import numpy as np

from tensorflow.keras.datasets import mnist

from sklearn.ensemble import ExtraTreesClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# ==========================================
# 1. MNIST 데이터 로드 & 전처리
# ==========================================
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_train_full = X_train_full.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test       = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0
y_train_full = y_train_full.astype(np.int8)
y_test       = y_test.astype(np.int8)

print("Full Train:", X_train_full.shape, "Test:", X_test.shape)

# ---- (1) 학습 속도용: train 일부만 사용 (예: 20,000개) ----
X_train_sub, _, y_train_sub, _ = train_test_split(
    X_train_full, y_train_full,
    train_size=20000,       # 필요하면 15000, 30000 등으로 조절
    random_state=42,
    stratify=y_train_full
)

print("Used Train subset:", X_train_sub.shape)

# ==========================================
# 2. 개별 기본 모델(base estimators)
# ==========================================
clf_et = ExtraTreesClassifier(
    n_estimators=50,         # (2) 트리 개수 줄여서 속도↓
    random_state=42,
    n_jobs=-1
)

clf_svc = SVC(
    kernel='linear',
    probability=True,        # 스태킹에서 predict_proba 사용
    random_state=42
)

clf_mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    max_iter=15,             # (2) epoch 줄여서 속도↓
    alpha=1e-4,
    random_state=42
)

# ==========================================
# 3. StackingClassifier 구성
# ==========================================
estimators = [
    ('et',  clf_et),
    ('svc', clf_svc),
    ('mlp', clf_mlp),
]

meta_model = LogisticRegression(max_iter=200)

stack_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    stack_method='predict_proba',  # 확률 기반 스태킹
    n_jobs=-1,
    passthrough=False,
    cv=3                           # (3) 기본 5-fold → 3-fold 로 줄여서 속도↓
)

# ==========================================
# 4. 스태킹 모델 학습
# ==========================================
print("\n[Train] StackingClassifier (subset + 3-fold CV)")
stack_clf.fit(X_train_sub, y_train_sub)

# ==========================================
# 5. 테스트 성능 평가
# ==========================================
test_pred_stack = stack_clf.predict(X_test)
acc_stack = accuracy_score(y_test, test_pred_stack)

print("\n=== Test Accuracy ===")
print("Stacking :", acc_stack)


Full Train: (60000, 784) Test: (10000, 784)
Used Train subset: (20000, 784)

[Train] StackingClassifier (subset + 3-fold CV)

=== Test Accuracy ===
Stacking : 0.9678
